# Introduction to the Multi-Agents



In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.16.0"
#r "nuget: Microsoft.SemanticKernel.Agents.Abstractions, 1.16.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.16.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.16.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.0-alpha"


Installed Packages Microsoft.SemanticKernel, 1.16.0 Microsoft.SemanticKernel.Agents.Abstractions, 1.16.0-alpha Microsoft.SemanticKernel.Agents.Core, 1.16.0-alpha Microsoft.SemanticKernel.Agents.OpenAI, 1.16.0-alpha Microsoft.SemanticKernel.Connectors.OpenAI, 1.16.0

In [2]:
#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;
using System.ComponentModel;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Agents.Chat;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
private const string OpenAIFunctionEnabledModel = "gpt-4o";

In [3]:
Kernel kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
                modelId: OpenAIFunctionEnabledModel,
                apiKey: apiKey)
            .Build();

In [4]:
// Lets given the Persona to Agents

string ProgamManager = """
    You are a program manager which will take the requirement and create a plan for creating app. Program Manager understands the 
    user requirements and form the detail documents with requirements and costing. 
""";

string SoftwareEngineer = """
   You are Software Engieer, and your goal is create web app using HTML and JavaScript by taking into consideration all
   the requirements given by Program Manager. 
""";

string ProjectManager = """
    You are manager which will review software engineer code, and make sure all client requirements are completed. 
    You are the guardian of quality, ensuring the final product meets all specifications and receives the green light for release.
     Once all client requirements are completed, you can approve the request by just responding "approve"
""";

In [5]:
#pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details

 ChatCompletionAgent ProgaramManagerAgent =
            new()
            {
                Instructions = ProgamManager,
                Name = "ProgaramManagerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent SoftwareEngineerAgent =
            new()
            {
                Instructions = SoftwareEngineer,
                Name = "SoftwareEngineerAgent",
                Kernel = kernel
            };

 ChatCompletionAgent ProjectManagerAgent =
            new()
            {
                Instructions = ProjectManager,
                Name = "ProjectManagerAgent",
                Kernel = kernel
            };

In [6]:
    #pragma warning disable SKEXP0110, SKEXP0001 // Rethrow to preserve stack details
    
    using System.Threading;
    using Microsoft.SemanticKernel.Agents.Chat;
    
    private sealed class ApprovalTerminationStrategy : TerminationStrategy
    {
        // Terminate when the final message contains the term "approve"
        protected override Task<bool> ShouldAgentTerminateAsync(Agent agent, IReadOnlyList<ChatMessageContent> history, CancellationToken cancellationToken)
            => Task.FromResult(history[history.Count - 1].Content?.Contains("approve", StringComparison.OrdinalIgnoreCase) ?? false);
    }

In [7]:
#pragma warning disable SKEXP0110, SKEXP0001 

AgentGroupChat chat =
    new(ProgaramManagerAgent, SoftwareEngineerAgent, ProjectManagerAgent)
    {
        ExecutionSettings =
            new()
            {
                TerminationStrategy =
                    new ApprovalTerminationStrategy()
                    {
                        Agents = [ProjectManagerAgent],
                        MaximumIterations = 6,
                    }
            }
};

In [9]:
#pragma warning disable SKEXP0110, SKEXP0001 
// Invoke chat and display messages.
string input = """

I want to develop app which will provide me selection for most common cocktails. Keep it very simple. And get final approval from manager.
""";

chat.AddChatMessage(new ChatMessageContent(AuthorRole.User, input));
Console.WriteLine($"# {AuthorRole.User}: '{input}'");

await foreach (var content in chat.InvokeAsync())
{
    Console.WriteLine($"# {content.Role} - {content.AuthorName ?? "*"}: '{content.Content}'");
}

# user: '
I want to develop app which will provide me selection for most common cocktails. Keep it very simple. And get final approval from manager.'
# assistant - ProgaramManagerAgent: 'Great! Let's start by outlining a simple app that provides users with a selection of the most common cocktails. This app will allow users to view a list of cocktails, and upon selection, see details such as ingredients and preparation method.

### Common Cocktails App Requirements:

#### 1. **Core Functionalities:**
   - A home screen displaying a list of common cocktails.
   - Ability to select a cocktail to view its details.
   - Details screen showing the ingredients and preparation method.

#### 2. **Design Requirements:**
   - Clean and intuitive UI.
   - Consistent styling for list and detail views.
   - Responsive design to work on various screen sizes.

#### 3. **Technical Specifications:**
   - Platform: iOS and/or Android.
   - Language: Swift for iOS, Kotlin for Android. Alternatively, Flutt